**Data Annotation for fine-tuning GPT**

In [ ]:
# import the libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from textblob import TextBlob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import the data

df = pd.read_csv(r"/content/drive/MyDrive/Sentiment Analysis using GPT/data/Bitcoin_tweets.csv")

df.tail(10)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
48573,H N,NaN,NaN,2021-02-20 16:48:16,1,8,10,False,2021-03-11 22:32:25,Buy #bitcoin !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,"['bitcoin', 'crypto', 'altcoins', 'BTC']",Twitter for iPhone,False
48574,DECRYPTOTECHNOLOGIST,NaN,DECRYPTOTECHNOLOGIST\n\nGithub - https://t.co/...,2021-02-07 20:14:41,59,389,757,False,2021-03-11 22:32:00,#BTC #bitcoin just broke 58k again - are you b...,"['BTC', 'bitcoin', 'QAnon', 'chinavirus', '4mo...",Twitter for Android,False
48575,S O N N Y,"Los Angeles, CA",#Crypto Investor + Crypto Trader #Bitcoin #BTC...,2012-11-03 22:31:45,12014,843,1225,False,2021-03-11 22:31:36,#Bitcoin Big Double pay again!!! Once it hit $...,['Bitcoin'],Twitter for Android,False
48576,Sokyu Honma,NaN,💯 FB 📰 $ETH #Bitcoin #Forex #DowJones #NASDAQ ...,2020-12-12 23:55:52,511,572,361,False,2021-03-11 22:31:27,#BTC #Bitcoin #FOREX\nNZD/USD forecast to 0.75...,"['BTC', 'Bitcoin', 'FOREX']",Zapier.com,False
48577,CPonCT,"Belfast, Northern Ireland","Medical Doctor, \nCrypto Dabbler, & \nFrequent...",2012-04-10 21:28:50,85,277,3188,False,2021-03-11 22:31:26,$BTC pop or drop?\n\n#volatility in 3...2...1....,"['volatility', 'BTC', 'Bitcoin', 'breakout', '...",Twitter for Android,False
48578,Jonbull,"New York, USA",Author of the book Jonbull's Stock Guide @Guid...,2010-03-15 03:38:08,2787,4275,6300,False,2021-03-11 22:30:23,Bill Gates advised against investing in #Bitco...,"['Bitcoin', 'BTC']",Twitter for iPhone,False
48579,BTCtoCAD,Canada,#Bitcoin Is King 👑\nBitcoin(BTC) to Canadian D...,2020-07-29 01:50:02,201,30,146,False,2021-03-11 22:30:04,#Bitcoin price in US Dollar #BTC $usd #BTCUSD ...,"['Bitcoin', 'BTC', 'BTCUSD', 'xbtusd']",NaN,False
48580,PiRoyot,NaN,Test Twitter Bot for Pi,2020-06-07 10:46:59,5,0,0,False,2021-03-11 22:30:02,Market Id : BTC-AUD\nLast Price :74260.91\n24 ...,"['btc', 'bitcoin']",HashTagTrendRoy,False
48581,Neozul,Milky way,#crypto #bitcoin #ethereum #ICT,2009-05-18 17:40:53,226,461,1299,False,2021-03-11 22:29:39,Bitcoin retesting ATH ❤️\n#Bitcoin #btc #Crypto,"['Bitcoin', 'btc', 'Crypto']",Twitter for iPhone,False
48582,BitcoinTracker,NaN,Tracking Bitcoin Prices and Moving Averages wi...,2021-02-17 21:05:57,54,8,8,False,2021-03-11 22:29:38,"$BTCUSD\nCurrent Bitcoin Price: $57,908\nDay H...",NaN,NaN,False


In [ ]:
df.describe()

,user_followers,user_friends,user_favourites
count,3.841600e+04,38416.000000,38416.000000
mean,5.965232e+03,1261.147256,6594.779857
std,5.600505e+04,6018.766404,19966.980339
min,0.000000e+00,0.000000,0.000000
25%,8.700000e+01,57.000000,72.000000
50%,4.200000e+02,245.000000,691.500000
75%,2.146000e+03,941.000000,4479.500000
max,3.977233e+06,701484.000000,488627.000000


In [ ]:
# remove nan value row in hastags column
df. dropna(subset = ["hashtags"], inplace=True)
df.tail(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
48577,CPonCT,"Belfast, Northern Ireland","Medical Doctor, \nCrypto Dabbler, & \nFrequent...",2012-04-10 21:28:50,85,277,3188,False,2021-03-11 22:31:26,$BTC pop or drop?\n\n#volatility in 3...2...1....,"['volatility', 'BTC', 'Bitcoin', 'breakout', '...",Twitter for Android,False
48578,Jonbull,"New York, USA",Author of the book Jonbull's Stock Guide @Guid...,2010-03-15 03:38:08,2787,4275,6300,False,2021-03-11 22:30:23,Bill Gates advised against investing in #Bitco...,"['Bitcoin', 'BTC']",Twitter for iPhone,False
48579,BTCtoCAD,Canada,#Bitcoin Is King 👑\nBitcoin(BTC) to Canadian D...,2020-07-29 01:50:02,201,30,146,False,2021-03-11 22:30:04,#Bitcoin price in US Dollar #BTC $usd #BTCUSD ...,"['Bitcoin', 'BTC', 'BTCUSD', 'xbtusd']",NaN,False
48580,PiRoyot,NaN,Test Twitter Bot for Pi,2020-06-07 10:46:59,5,0,0,False,2021-03-11 22:30:02,Market Id : BTC-AUD\nLast Price :74260.91\n24 ...,"['btc', 'bitcoin']",HashTagTrendRoy,False
48581,Neozul,Milky way,#crypto #bitcoin #ethereum #ICT,2009-05-18 17:40:53,226,461,1299,False,2021-03-11 22:29:39,Bitcoin retesting ATH ❤️\n#Bitcoin #btc #Crypto,"['Bitcoin', 'btc', 'Crypto']",Twitter for iPhone,False


In [ ]:
# total value size
print("Tweet Size: ", len(df))

Tweet Size:  48583


In [ ]:
# get only texts
df = df[['text']]
df.columns = ['tweets']

In [ ]:
# create a function to clean the tweets
def cleanTwt(twt):
    twt = re.sub("#bitcoin", 'bitcoin', twt) # removes the '#' from bitcoin
    twt = re.sub("#Bitcoin", 'Bitcoin', twt) # removes the '#' from Bitcoin
    twt = re.sub('#[A-Za-z0-9]+', '', twt) # removes any string with a '#'
    twt = re.sub('\\n', '', twt) # removes the '\n' string
    twt = re.sub('https:\/\/\S+', '', twt) # removes any hyperlinks
    return twt

In [ ]:
df['cleaned_tweets'] = df['tweets'].apply(cleanTwt)
df

,tweets,cleaned_tweets
0,Blue Ridge Bank shares halted by NYSE after #b...,Blue Ridge Bank shares halted by NYSE after bi...
1,"😎 Today, that's this #Thursday, we will do a ""...","😎 Today, that's this , we will do a ""🎬 Take 2""..."
2,"Guys evening, I have read this article about B...","Guys evening, I have read this article about B..."
3,$BTC A big chance in a billion! Price: \487264...,$BTC A big chance in a billion! Price: \487264...
4,This network is secured by 9 508 nodes as of t...,This network is secured by 9 508 nodes as of t...
...,...,...
48578,Bill Gates advised against investing in #Bitco...,Bill Gates advised against investing in Bitcoi...
48579,#Bitcoin price in US Dollar #BTC $usd #BTCUSD ...,Bitcoin price in US Dollar $usd $XBT BTC Pr...
48580,Market Id : BTC-AUD\nLast Price :74260.91\n24 ...,Market Id : BTC-AUDLast Price :74260.9124 Hour...
48581,Bitcoin retesting ATH ❤️\n#Bitcoin #btc #Crypto,Bitcoin retesting ATH ❤️Bitcoin


In [ ]:
# create a function to get subjectivity
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity

# create a function to get the polarity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity

# create two new columns called "Subjectivity" & "Polarity"
df['subjectivity'] = df['cleaned_tweets'].apply(getSubjectivity)
df['polarity'] = df['cleaned_tweets'].apply(getPolarity)

In [ ]:
df.head(5)

,tweets,cleaned_tweets,subjectivity,polarity
0,Blue Ridge Bank shares halted by NYSE after #b...,Blue Ridge Bank shares halted by NYSE after bi...,0.1,0.0
1,"😎 Today, that's this #Thursday, we will do a ""...","😎 Today, that's this , we will do a ""🎬 Take 2""...",0.0,0.0
2,"Guys evening, I have read this article about B...","Guys evening, I have read this article about B...",0.0,0.0
3,$BTC A big chance in a billion! Price: \487264...,$BTC A big chance in a billion! Price: \487264...,0.1,0.0
4,This network is secured by 9 508 nodes as of t...,This network is secured by 9 508 nodes as of t...,0.1,0.0


In [ ]:
# create a function get the sentiment text
def getSentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [ ]:
# create a column to store the text sentiment
df['sentiment'] = df['polarity'].apply(getSentiment)

# show the data
df.head()

,tweets,cleaned_tweets,subjectivity,polarity,sentiment
0,Blue Ridge Bank shares halted by NYSE after #b...,Blue Ridge Bank shares halted by NYSE after bi...,0.1,0.0,neutral
1,"😎 Today, that's this #Thursday, we will do a ""...","😎 Today, that's this , we will do a ""🎬 Take 2""...",0.0,0.0,neutral
2,"Guys evening, I have read this article about B...","Guys evening, I have read this article about B...",0.0,0.0,neutral
3,$BTC A big chance in a billion! Price: \487264...,$BTC A big chance in a billion! Price: \487264...,0.1,0.0,neutral
4,This network is secured by 9 508 nodes as of t...,This network is secured by 9 508 nodes as of t...,0.1,0.0,neutral


In [ ]:
data.reset_index()

NameError: ignored

In [ ]:
data.to_csv('/content/drive/MyDrive/Sentiment Analysis using GPT/data/twtdata.csv',index=False)